In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [35]:
torch.manual_seed(1) #똑같이 결과를 재현해주기 위해 토치 씨드를 준다.

x_data = [[1,2],[2,3],[3,1],[4,3],[5,3],[6,2]] #(6,2)
y_data = [[0],[0],[0],[1],[1],[1]] #(6,)

x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

#### class로 만들기

In [39]:
class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(2, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        return self.sigmoid(self.linear(x))

model = BinaryClassifier()

optimizer = optim.SGD(model.parameters(), lr = 1)
nb_epochs = 100
for epoch in range(nb_epochs + 1):
    hypothesis = model(x_train)
    cost = F.binary_cross_entropy(hypothesis, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 10 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5])
        correct_prediction = prediction.float() == y_train
        accuracy = correct_prediction.sum().item() / len(correct_prediction)
        print('Epoch {:4d} / {} Cost {:.6f} Accuracy {:2.2f}%'.format(
        epoch, nb_epochs, cost.item(), accuracy * 100))

Epoch    0 / 100 Cost 0.734527 Accuracy 50.00%
Epoch   10 / 100 Cost 0.446570 Accuracy 66.67%
Epoch   20 / 100 Cost 0.448868 Accuracy 66.67%
Epoch   30 / 100 Cost 0.375859 Accuracy 83.33%
Epoch   40 / 100 Cost 0.318583 Accuracy 83.33%
Epoch   50 / 100 Cost 0.268096 Accuracy 83.33%
Epoch   60 / 100 Cost 0.222295 Accuracy 100.00%
Epoch   70 / 100 Cost 0.183465 Accuracy 100.00%
Epoch   80 / 100 Cost 0.158036 Accuracy 100.00%
Epoch   90 / 100 Cost 0.144541 Accuracy 100.00%
Epoch  100 / 100 Cost 0.134652 Accuracy 100.00%


#### 그냥만들기

In [37]:
print('e^1 equals: ', torch.exp(torch.FloatTensor([1])))
W = torch.zeros((2,1), requires_grad = True) #d가 2였으므로 2,1
#requires_grad = True: gradient를 배울거야
b = torch.zeros(1, requires_grad = True)
#다 zero로 주었음

optimizer = optim.SGD([W, b], lr = 1) #SGD를 가지고 w와 b를 업데이트할거야
#losses = -(y_train * torch.log(hypothesis) + (1- y_train) * torch.log(1 - hypothesis))


nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    #hypothesis = 1 / (1 + torch.exp(-(x_train.matmul(W) + b))) #h(x)를 계산(시그모이드 함수)
    hypothesis = torch.sigmoid(x_train.matmul(W) + b)
    cost = F.binary_cross_entropy(hypothesis, y_train)
    
    optimizer.zero_grad() #기존에 혹시나 구해놓은 gradient가 있으면 0으로 초기화
    #이걸 안하면 기존의 gradient에 더하게 되니까 꼭 해야함
    cost.backward() #backpropagation 수행
    optimizer.step() #cost값을 minimize하는 방향으로 w와 b를 업데이트
    
    if epoch % 100 == 0:
        print('Epoch {:4d} / {} Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()
        ))

e^1 equals:  tensor([2.7183])
Epoch    0 / 1000 Cost: 0.693147
Epoch  100 / 1000 Cost: 0.134722
Epoch  200 / 1000 Cost: 0.080643
Epoch  300 / 1000 Cost: 0.057900
Epoch  400 / 1000 Cost: 0.045300
Epoch  500 / 1000 Cost: 0.037261
Epoch  600 / 1000 Cost: 0.031672
Epoch  700 / 1000 Cost: 0.027556
Epoch  800 / 1000 Cost: 0.024394
Epoch  900 / 1000 Cost: 0.021888
Epoch 1000 / 1000 Cost: 0.019852


#### 검증

In [38]:
hypothesis = torch.sigmoid(x_train.matmul(W) + b)
print(hypothesis[:5])

prediction = hypothesis >= torch.FloatTensor([0.5])
print(prediction[:5])
print(y_train[:5])

correct_prediction = prediction.float() == y_train
print(correct_prediction)

tensor([[2.7648e-04],
        [3.1608e-02],
        [3.8977e-02],
        [9.5622e-01],
        [9.9823e-01]], grad_fn=<SliceBackward>)
tensor([[False],
        [False],
        [False],
        [ True],
        [ True]])
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.]])
tensor([[True],
        [True],
        [True],
        [True],
        [True],
        [True]])


#### 새로 파일 로드

In [41]:
#Evaluation
import numpy as np
xy = np.loadtxt('data-03-diabetes.csv', delimiter = ',', dtype = np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

print(x_train[0:5])
print(y_train[0:5])

tensor([[-0.2941,  0.4874,  0.1803, -0.2929,  0.0000,  0.0015, -0.5312, -0.0333],
        [-0.8824, -0.1457,  0.0820, -0.4141,  0.0000, -0.2072, -0.7669, -0.6667],
        [-0.0588,  0.8392,  0.0492,  0.0000,  0.0000, -0.3055, -0.4927, -0.6333],
        [-0.8824, -0.1055,  0.0820, -0.5354, -0.7778, -0.1624, -0.9240,  0.0000],
        [ 0.0000,  0.3769, -0.3443, -0.2929, -0.6028,  0.2846,  0.8873, -0.6000]])
tensor([[0.],
        [1.],
        [0.],
        [1.],
        [0.]])


In [42]:
W = torch.zeros((8, 1), requires_grad = True)
b = torch.zeros(1, requires_grad = True)

optimizer = optim.SGD([W, b], lr = 1)

nb_epochs = 100
for epoch in range(nb_epochs + 1):
    hypothesis = torch.sigmoid(x_train.matmul(W) + b)
    cost = -(y_train * torch.log(hypothesis) + (1-y_train) * torch.log(1 - hypothesis)).mean()
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 10 == 0:
        print('Epoch {:4d} / {} Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()))

Epoch    0 / 100 Cost: 0.693148
Epoch   10 / 100 Cost: 0.572727
Epoch   20 / 100 Cost: 0.539493
Epoch   30 / 100 Cost: 0.519708
Epoch   40 / 100 Cost: 0.507066
Epoch   50 / 100 Cost: 0.498539
Epoch   60 / 100 Cost: 0.492549
Epoch   70 / 100 Cost: 0.488208
Epoch   80 / 100 Cost: 0.484985
Epoch   90 / 100 Cost: 0.482543
Epoch  100 / 100 Cost: 0.480661


In [43]:
W = torch.zeros((8,1), requires_grad = True)
b = torch.zeros(1, requires_grad = True)
optimizer = optim.SGD([W,b], lr = 1)

nb_epochs = 100
for epoch in range(nb_epochs + 1):
    hypothesis = torch.sigmoid(x_train.matmul(W) + b)
    cost = F.binary_cross_entropy(hypothesis, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 10 == 0:
        print('Epoch {:4d} / {} Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()))

Epoch    0 / 100 Cost: 0.693147
Epoch   10 / 100 Cost: 0.572727
Epoch   20 / 100 Cost: 0.539494
Epoch   30 / 100 Cost: 0.519708
Epoch   40 / 100 Cost: 0.507065
Epoch   50 / 100 Cost: 0.498539
Epoch   60 / 100 Cost: 0.492549
Epoch   70 / 100 Cost: 0.488208
Epoch   80 / 100 Cost: 0.484985
Epoch   90 / 100 Cost: 0.482543
Epoch  100 / 100 Cost: 0.480661


In [50]:
hypothesis = torch.sigmoid(x_train.matmul(W) + b)
print(hypothesis[:5])

prediction = hypothesis >= torch.FloatTensor([0.5])
print(prediction.float()[:5])
print(y_train[:5])

correct_prediction = prediction.float() == y_train
print(correct_prediction.float()[:5])

accuracy = correct_prediction.sum().item() / len(correct_prediction)
print('The model has an accuracy of {:2.2f}% for the training set.'.format(accuracy * 100))

tensor([[0.4103],
        [0.9242],
        [0.2300],
        [0.9411],
        [0.1772]], grad_fn=<SliceBackward>)
tensor([[0.],
        [1.],
        [0.],
        [1.],
        [0.]])
tensor([[0.],
        [1.],
        [0.],
        [1.],
        [0.]])
tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.]])
The model has an accuracy of 76.68% for the training set.


In [51]:
class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(8, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        return self.sigmoid(self.linear(x))

model = BinaryClassifier()

optimizer = optim.SGD(model.parameters(), lr = 1)

nb_epochs = 100
for epoch in range(nb_epochs + 1):
    hypothesis = model(x_train)
    cost = F.binary_cross_entropy(hypothesis, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 10 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5])
        correct_prediction = prediction.float() == y_train
        accuracy = correct_prediction.sum().item() / len(correct_prediction)
        print('Epoch {:4d} / {} Cost: {:.6f} Accuracy {:2.2f}%'.format(
        epoch, nb_epochs, cost.item(), accuracy * 100))

Epoch    0 / 100 Cost: 0.803927 Accuracy 34.52%
Epoch   10 / 100 Cost: 0.570361 Accuracy 67.19%
Epoch   20 / 100 Cost: 0.538137 Accuracy 72.60%
Epoch   30 / 100 Cost: 0.518806 Accuracy 75.89%
Epoch   40 / 100 Cost: 0.506408 Accuracy 77.08%
Epoch   50 / 100 Cost: 0.498033 Accuracy 77.21%
Epoch   60 / 100 Cost: 0.492148 Accuracy 77.73%
Epoch   70 / 100 Cost: 0.487886 Accuracy 77.34%
Epoch   80 / 100 Cost: 0.484723 Accuracy 76.55%
Epoch   90 / 100 Cost: 0.482329 Accuracy 76.68%
Epoch  100 / 100 Cost: 0.480485 Accuracy 76.81%
